<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=9ebdf1c95aa81fac156faa796e6af4be51ccf54ef1ae2aa9af8b06317ded2b19
  Stored in directory: c:\users\tejal\appdata\local\pip\cache\wheels\01\46\3b\e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [3]:
import folium
import wget
import pandas as pd

In [5]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [7]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

Now, you can take a look at what are the coordinates for each site.


In [9]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [13]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [15]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [21]:
import folium
from folium.features import DivIcon

# Initialize the map centered at NASA Johnson Space Center with a zoom level of 5
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each row in the launch_sites_df to add a circle for each launch site
for idx, row in launch_sites_df.iterrows():
    # Get the coordinates for the current launch site
    coordinate = [row['Lat'], row['Long']]
    
    # Create a circle with a radius of 1000 meters around the launch site
    circle = folium.Circle(
        coordinate, 
        radius=1000, 
        color='#d35400', 
        fill=True, 
        fill_color='#f39c12'
    ).add_child(folium.Popup(row['Launch Site']))  # Add popup with site name
    
    # Add the circle to the map
    site_map.add_child(circle)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [23]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [27]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [29]:
# Define a function to assign marker colors based on the 'class' value
def get_marker_color(class_value):
    if class_value == 1:
        return 'green'
    else:
        return 'red'

# Apply the function to create the 'marker_color' column
spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)

# Display the updated DataFrame with the 'marker_color' column
spacex_df[['Launch Site', 'Lat', 'Long', 'class', 'marker_color']].head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [31]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [35]:
spacex_df.columns


Index(['Launch Site', 'Lat', 'Long', 'class', 'marker_color'], dtype='object')

In [44]:
# Add markers with launch outcomes
for index, record in spacex_df.iterrows():
    # Define coordinates
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']

    # Create a marker with a custom icon color based on launch outcome
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=marker_color, icon_color='white'),
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if marker_color == 'green' else 'Failure'}"
    )

    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Add marker_cluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map


In [39]:
# Calculate success rate for each launch site
success_rate = spacex_df.groupby('Launch Site')['class'].mean()

# Print success rates for each site
print(success_rate)


Launch Site
CCAFS LC-40     0.269231
CCAFS SLC-40    0.428571
KSC LC-39A      0.769231
VAFB SLC-4E     0.400000
Name: class, dtype: float64


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [46]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [54]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance
    print (distance)

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [56]:
import folium
from folium.plugins import MousePosition
from math import sin, cos, sqrt, atan2, radians

# Function to calculate distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Radius of Earth in km

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Coordinates of KSC LC-39A Launch Site
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Coordinates of the closest coastline (use MousePosition plugin to get this)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print("Distance to coastline:", round(distance_coastline, 2), "km")

# Create map centered at launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add Mouse Position plugin to map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Add launch site marker
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='KSC LC-39A Launch Site',
    icon=folium.Icon(color='green')
).add_to(site_map)

# Add coastline marker
folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup='Closest Coastline',
    icon=folium.Icon(color='blue')
).add_to(site_map)

# Add a circle to visualize the distance
folium.Circle(
    location=[launch_site_lat, launch_site_lon],
    radius=distance_coastline * 1000,  # in meters
    color='blue',
    fill=True,
    fill_opacity=0.1
).add_to(site_map)

# Display the map
site_map


Distance to coastline: 7.43 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [58]:
import folium
from folium import Marker
from folium import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Function to calculate the distance between two points based on Lat, Lon
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance

# Initialize the map centered around KSC LC-39A
site_map = folium.Map(location=[28.573255, -80.646895], zoom_start=12)

# Coordinates of KSC LC-39A (launch site) and closest coastline (from previous step)
launch_site_lat = 28.573255
launch_site_lon = -80.646895
coastline_lat = 28.56367  # Example coordinates for coastline point
coastline_lon = -80.57163

# Calculate the distance to the coastline
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create and add a marker for the coastline point
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance)
    )
)

# Add the marker to the map
distance_marker.add_to(site_map)

# Add the launch site marker to the map (for reference)
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='Launch Site: KSC LC-39A',
    icon=folium.Icon(color='green')
).add_to(site_map)

# Show the map
site_map


_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [60]:
import folium
from folium import Marker
from folium import DivIcon, PolyLine
from math import sin, cos, sqrt, atan2, radians

# Function to calculate the distance between two points based on Lat, Lon
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance

# Initialize the map centered around KSC LC-39A
site_map = folium.Map(location=[28.573255, -80.646895], zoom_start=12)

# Coordinates of KSC LC-39A (launch site) and closest coastline (from previous step)
launch_site_lat = 28.573255
launch_site_lon = -80.646895
coastline_lat = 28.56367  # Example coordinates for coastline point
coastline_lon = -80.57163

# Calculate the distance to the coastline
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create and add a marker for the coastline point
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance)
    )
)
distance_marker.add_to(site_map)

# Add the launch site marker to the map (for reference)
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='Launch Site: KSC LC-39A',
    icon=folium.Icon(color='green')
).add_to(site_map)

# Create a PolyLine between the launch site and the coastline point
coordinates = [(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)]
lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')

# Add the PolyLine to the map
site_map.add_child(lines)

# Show the map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [62]:
# Create a marker with distance to a closest city, railway, highway, etc.
import folium
from folium import Marker
from folium import DivIcon, PolyLine
from folium.plugins import MousePosition
from math import sin, cos, sqrt, atan2, radians

# Function to calculate the distance between two points based on Lat, Lon
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance

# Initialize the map centered around KSC LC-39A
site_map = folium.Map(location=[28.573255, -80.646895], zoom_start=12)

# Coordinates of KSC LC-39A (launch site)
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Enable MousePosition to get coordinates of points (e.g., city, railway, highway, etc.)
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Example coordinates for a city (change this to actual city coordinates you find)
city_lat = 28.5383
city_lon = -80.6480

# Calculate the distance to the city
distance_to_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create and add a marker for the city point
city_marker = folium.Marker(
    location=[city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#3498db;"><b>{:10.2f} KM</b></div>'.format(distance_to_city)
    )
)
city_marker.add_to(site_map)

# Add the launch site marker to the map
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='Launch Site: KSC LC-39A',
    icon=folium.Icon(color='green')
).add_to(site_map)

# Create a PolyLine between the launch site and the city
coordinates_city = [(launch_site_lat, launch_site_lon), (city_lat, city_lon)]
lines_city = folium.PolyLine(locations=coordinates_city, weight=1, color='red')

# Add the PolyLine to the map
site_map.add_child(lines_city)

# Example coordinates for a railway (change this to actual railway coordinates you find)
railway_lat = 28.5700
railway_lon = -80.6100

# Calculate the distance to the railway
distance_to_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# Create and add a marker for the railway point
railway_marker = folium.Marker(
    location=[railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#e74c3c;"><b>{:10.2f} KM</b></div>'.format(distance_to_railway)
    )
)
railway_marker.add_to(site_map)

# Create a PolyLine between the launch site and the railway
coordinates_railway = [(launch_site_lat, launch_site_lon), (railway_lat, railway_lon)]
lines_railway = folium.PolyLine(locations=coordinates_railway, weight=1, color='green')

# Add the PolyLine to the map
site_map.add_child(lines_railway)

# Example coordinates for a highway (change this to actual highway coordinates you find)
highway_lat = 28.5800
highway_lon = -80.6000

# Calculate the distance to the highway
distance_to_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Create and add a marker for the highway point
highway_marker = folium.Marker(
    location=[highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#f39c12;"><b>{:10.2f} KM</b></div>'.format(distance_to_highway)
    )
)
highway_marker.add_to(site_map)

# Create a PolyLine between the launch site and the highway
coordinates_highway = [(launch_site_lat, launch_site_lon), (highway_lat, highway_lon)]
lines_highway = folium.PolyLine(locations=coordinates_highway, weight=1, color='purple')

# Add the PolyLine to the map
site_map.add_child(lines_highway)

# Show the map
site_map



In [64]:
# Draw a line between the marker to the launch site

import folium
from folium import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Launch site coordinates
launch_site = [28.573255, -80.646895]

# Example marker: coastline coordinates
coastline = [28.56367, -80.57163]

# Calculate distance
distance = calculate_distance(launch_site[0], launch_site[1], coastline[0], coastline[1])

# Create map centered at launch site
site_map = folium.Map(location=launch_site, zoom_start=12)

# Add launch site marker
folium.Marker(launch_site, popup='Launch Site', icon=folium.Icon(color='green')).add_to(site_map)

# Add coastline marker with distance label
folium.Marker(
    coastline,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance),
    )
).add_to(site_map)

# Draw PolyLine from launch site to coastline
folium.PolyLine([launch_site, coastline], color='blue', weight=2.5, opacity=1).add_to(site_map)

# Display the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
